# Preliminaries

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
config = {
    'outputs':['/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-1.npy',
               '/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-2.npy',
               '/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-3.npy',
               '/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-4.npy',
               '/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-5.npy',
               '/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-6.npy'],
    'positive_pairs':'/gdrive/MyDrive/shared_space/commonvoice-farsi/positive-pairs.pkl',
    'negative_pairs':'/gdrive/MyDrive/shared_space/commonvoice-farsi/negative-pairs.pkl',
    'path_to_part':'/gdrive/MyDrive/shared_space/commonvoice-farsi/path-to-part.pkl',
    'path_to_index':'/gdrive/MyDrive/shared_space/commonvoice-farsi/path-to-index.pkl',
    'models_path':'/gdrive/MyDrive/arman/verification-models/',
    'num_epochs':200,
    'learning_rate':1e-1,
    'batch_size':32,
    'train_percentage':90
}

In [3]:
import numpy as np
import pickle as pkl
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
from sklearn.model_selection import train_test_split

# Loading the Data

In [4]:
# outputs = [np.load(f'/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-{i}.npy', allow_pickle=True) for i in range(1,7)]

In [5]:
# outputs[0].shape

In [6]:
# outputs[3].shape

In [7]:
# with open(config['positive_pairs'], 'rb') as file:
#   pos_pairs = pkl.load(file)
# with open(config['negative_pairs'], 'rb') as file:
#   neg_pairs = pkl.load(file)
# with open(config['path_to_part'], 'rb') as file:
#   path_to_part = pkl.load(file)
# with open(config['path_to_index'], 'rb') as file:
#   path_to_index = pkl.load(file)

In [8]:
# pos_pairs[0]

In [9]:
def pair_to_array(pair):
  first_array = outputs[int(path_to_part[pair[0]])-1][path_to_index[pair[0]]]
  second_array = outputs[int(path_to_part[pair[1]])-1][path_to_index[pair[1]]]
  return first_array, second_array

In [10]:
# pair_to_array(pos_pairs[0])[0].shape

In [11]:
# pair_to_array(pos_pairs[0])[1].shape

In [12]:
# pos_arrays = np.array([pair_to_array(pair) for pair in pos_pairs])
# neg_arrays = np.array([pair_to_array(pair) for pair in neg_pairs])

In [13]:
# pos_arrays.shape

In [14]:
# neg_arrays.shape

In [15]:
# max_frames = 0
# all_lengths = []
# for item in pos_arrays:
#   all_lengths.append(item[0].shape[0])
#   all_lengths.append(item[1].shape[0])
#   curr_max = max(item[0].shape[0], item[1].shape[0])
#   if curr_max > max_frames:
#     max_frames = curr_max

In [16]:
# max_frames

In [17]:
# sum(all_lengths)/len(all_lengths)

In [18]:
# pos_arrays[0][0].shape

In [19]:
# pos_arrays[0][0][:200].shape

In [20]:
def fix_length(arr, desired_length=500):
  if arr.shape[0]<desired_length:
    tmp = np.zeros((desired_length-arr.shape[0], 40))
    output = np.concatenate((arr, tmp), axis=0)
  elif arr.shape[0]>desired_length:
    output = arr[:desired_length]
  else:
    output = arr
  return output

In [21]:
# outputs = [np.load(f'/gdrive/MyDrive/shared_space/commonvoice-farsi/fbanks-{i}.npy', allow_pickle=True) for i in range(1,7)]
# with open(config['positive_pairs'], 'rb') as file:
#   pos_pairs = pkl.load(file)
# with open(config['negative_pairs'], 'rb') as file:
#   neg_pairs = pkl.load(file)
# with open(config['path_to_part'], 'rb') as file:
#   path_to_part = pkl.load(file)
# with open(config['path_to_index'], 'rb') as file:
#   path_to_index = pkl.load(file)
# pos_arrays = np.array([pair_to_array(pair) for pair in pos_pairs])
# neg_arrays = np.array([pair_to_array(pair) for pair in neg_pairs])

# tmp_pos = pos_arrays.tolist()
# tmp_pos = np.array([[fix_length(item[0]), fix_length(item[1])] for item in tmp_pos])
# tmp_neg = neg_arrays.tolist()
# tmp_neg = np.array([[fix_length(item[0]), fix_length(item[1])] for item in tmp_neg])
# data = np.concatenate((tmp_pos, tmp_neg), axis=0)
# labels = np.array([1 if i < pos_arrays.shape[0] else 0 for i in range(data.shape[0])])
# np.save('/gdrive/MyDrive/shared_space/commonvoice-farsi/data.npy', data, allow_pickle=True)
# np.save('/gdrive/MyDrive/shared_space/commonvoice-farsi/labels.npy', labels, allow_pickle=True)

In [22]:
data = np.load('/gdrive/MyDrive/shared_space/commonvoice-farsi/data.npy', allow_pickle=True)
labels = np.load('/gdrive/MyDrive/shared_space/commonvoice-farsi/labels.npy', allow_pickle=True)

# Splitting the data into training, testing and validation (development) parts

In [23]:
f_train, f_rem, l_train, l_rem = train_test_split(data, labels, test_size=1-config['train_percentage']/100, random_state=50)
f_test, f_dev, l_test, l_dev = train_test_split(f_rem, l_rem, test_size=0.5, random_state=50)

In [24]:
print(f'train features: {f_train.shape}, dev features: {f_dev.shape}, test features: {f_test.shape}')

train features: (19197, 2, 500, 40), dev features: (1067, 2, 500, 40), test features: (1067, 2, 500, 40)


In [25]:
print(f'train labels: {len(l_train)}, dev labels: {len(l_dev)}, test labels: {len(l_test)}')

train labels: 19197, dev labels: 1067, test labels: 1067


# Converting Everything to Tensors

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [27]:
class MyDataset(Dataset):
    def __init__(self, features, labels, device):
        self.features = torch.from_numpy(features).to(device)
        self.labels = torch.from_numpy(np.array(labels).reshape(-1, 1)).to(device)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [28]:
train_dataset = MyDataset(f_train, l_train, device)
test_dataset = MyDataset(f_test, l_test, device)
dev_dataset = MyDataset(f_dev, l_dev, device)

In [29]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=config['batch_size'], shuffle=True)

In [30]:
train_dataset[0][0].shape

torch.Size([2, 500, 40])

# Neural Network Architecture

In [31]:
class my_neural_net(torch.nn.Module):
    def __init__(self):
        super(my_neural_net, self).__init__() 
        self.lstm = nn.LSTM(40, 20, 1, batch_first=True)
        self.flatten = torch.nn.Flatten()
        self.seq_layer = torch.nn.Sequential( 
            nn.Linear(40, 20),
            nn.ReLU(),
            nn.Linear(20, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = x.transpose(1,0)
        first_seq = x[0].to(torch.float32)
        second_seq = x[1].to(torch.float32)
        first_lstm_out = self.lstm(first_seq)[1][0]
        second_lstm_out = self.lstm(second_seq)[1][0]
        # return first_lstm_out, second_lstm_out
        concatenated = torch.cat((first_lstm_out, second_lstm_out), 2)
        # return concatenated
        concatenated = torch.squeeze(concatenated)
        # return concatenated
        # flattened = self.flatten(concatenated)
        output = self.seq_layer(concatenated)
        return output

In [32]:
my_nn = my_neural_net()
my_nn = my_nn.to(device)

In [44]:
train_dataset[:5][0].shape

torch.Size([5, 2, 500, 40])

In [33]:
(my_nn(train_dataset[:5][0])).shape

torch.Size([5, 1])

In [34]:
my_nn(train_dataset[:3][0])

tensor([[0.5834],
        [0.5916],
        [0.5666]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [35]:
l_train.shape

(19197,)

# Training

In [36]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(my_nn.parameters(), lr=config['learning_rate'])

In [37]:
def output_to_label(out):
    dist_to_0 = abs(out)
    dist_to_1 = abs(out-1)
    if dist_to_0 <= dist_to_1:
        return 0
    else:
        return 1

In [38]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        labels = labels.to(torch.float32)
        pred = model(features).to(torch.float32)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"\r Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]", end=" ")


def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            labels = labels.to(torch.float32)
            pred = model(features).to(torch.float32)
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.cpu().numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.cpu().numpy().squeeze()) 
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==o else 0 for (p, o) in zip(pred_lst, output_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")

In [39]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, my_nn, loss_fn, optimizer, epoch_num)
    test_loop(dev_dataloader, my_nn, loss_fn, epoch_num, 'Development/Validation')

 Epoch 200 - Development/Validation Error: Accuracy: 83.8%, Avg loss: 0.013110 

In [40]:
test_loop(train_dataloader, my_nn, loss_fn, epoch_num, 'Training')

 Epoch 200 - Training Error: Accuracy: 85.5%, Avg loss: 0.010987 

In [41]:
test_loop(test_dataloader, my_nn, loss_fn, epoch_num, 'Test')

 Epoch 200 - Test Error: Accuracy: 84.3%, Avg loss: 0.011854 

In [ ]:
my_nn

In [42]:
torch.save(my_nn.state_dict(), config['models_path']+"neural_net4.pth")